# 1. Information about the submission

## 1.1 Name and number of the assignment

#### #3 Text Detoxification

## 1.2 Student name

Ivan Gurev

## 1.3 Codalab user ID

IvanGurev

## 1.4 Additional comments

***Enter here** any additional comments which you would like to communicate to a TA who is going to grade this work not related to the content of your submission.*

# 2. Technical Report

*Use Section 2 to describe results of your experiments as you would do writing a paper about your results. DO NOT insert code in this part. Only insert plots and tables summarizing results as needed. Use formulas if needed do described your methodology. The code is provided in Section 3.*

## 2.1 Methodology

In this task I was aimed to provide model which is able to detoxify input text.

### Preprocessing
The initial dataset is tsv file, which contains toxic comments `toxic_comment` and several neutral rephrases `neutral_comment1`,`neutral_comment2`,`neutral_comment3` for them . Since amount of rephrases for each toxic comment varies, the structure of the file was remade to `toxic_comment`, `neutral_comment`. After that standart pipeline for training (`Dataset`, `Data_collator`) was done.


### Modelling

As a detoxificator `T5ForConditionalGeneration` ru_base was used. Model was pretrained by **ai-forever**.


## 2.2 Discussion of results

After trainnig for 10 epochs model shows appropriate results breaking the baseline. I suppose results can be improved by adding text augmentations and training for more epochs.

Method | STA |  MP | Fluency | Joint Score | ChrF1
--- | --- | --- | --- | --- | ---
Baseline | 0.56 | 0.89 | 0.85 | 0.41 | 0.53
T5 on train set | 0.72 | 0.81 | 0.84| 0.48 | 0.56
T5 on test set  | 0.75 | 0.83 | 0.81 | 0.51 | 0.57



# 3. Code

*Enter here all code used to produce your results submitted to Codalab. Add some comments and subsections to navigate though your solution.*

*In this part you are expected to develop yourself a solution of the task and provide a reproducible code:*
- *Using Python 3;*
- *Contains code for installation of all dependencies;*
- *Contains code for downloading of all the datasets used*;
- *Contains the code for reproducing your results (in other words, if a tester downloads your notebook she should be able to run cell-by-cell the code and obtain your experimental results as described in the methodology section)*.


*As a result, you code will be graded according to these criteria:*
- ***Readability**: your code should be well-structured preferably with indicated parts of your approach (Preprocessing, Model training, Evaluation, etc.).*
- ***Reproducibility**: your code should be reproduced without any mistakes with “Run all” mode (obtaining experimental part).*


## 3.1 Requirements

In [ ]:
!pip install transformers sentencepiece sacrebleu
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import (T5ForConditionalGeneration,
                          T5Tokenizer,
                          Seq2SeqTrainer,
                          Seq2SeqTrainingArguments,
                      )
import os
from sklearn.utils import shuffle
import pandas as pd
import torch
import re
from torch.utils.data import Dataset
from transformers.file_utils import cached_property
from typing import Tuple
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 3.2 Download the data

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/s-nlp/russe_detox_2022/main/data/input/train.tsv',sep='	', header=0)
test = pd.read_csv('https://raw.githubusercontent.com/s-nlp/russe_detox_2022/main/data/input/test.tsv',sep='	', header=0)
df.to_csv('drive/MyDrive/t5_detox/train.tsv', sep = '\t')
test.to_csv('drive/MyDrive/t5_detox/test.tsv', sep = '\t')



## 3.3 Preprocessing

In [ ]:
df = df.fillna('')

df_train_toxic = []
df_train_neutral = []

for index, row in df.iterrows():
    references = row[['neutral_comment1', 'neutral_comment2', 'neutral_comment3']].tolist()

    for reference in references:
        if len(reference) > 0:
            df_train_toxic.append(row['toxic_comment'])
            df_train_neutral.append(reference)
        else:
            break


df = pd.DataFrame({
    'toxic_comment': df_train_toxic,
    'neutral_comment': df_train_neutral
})

df = shuffle(df)


class PairsDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        assert idx < len(self.x['input_ids'])
        item = {key: val[idx] for key, val in self.x.items()}
        item['decoder_attention_mask'] = self.y['attention_mask'][idx]
        item['labels'] = self.y['input_ids'][idx]
        return item

    @property
    def n(self):
        return len(self.x['input_ids'])

    def __len__(self):
        return self.n # * 2

from typing import List, Dict, Union

class DataCollatorWithPadding:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.tokenizer.pad(
            features,
            padding=True,
        )
        ybatch = self.tokenizer.pad(
            {'input_ids': batch['labels'], 'attention_mask': batch['decoder_attention_mask']},
            padding=True,
        )
        batch['labels'] = ybatch['input_ids']
        batch['decoder_attention_mask'] = ybatch['attention_mask']

        return {k: torch.tensor(v) for k, v in batch.items()}




## 3.4 My method of text processing

In [ ]:
class Args:
    def __init__(self):
        self.model_name_or_path = 'ai-forever/ruT5-base'
        self.model_class = T5ForConditionalGeneration
        self.tokenizer_class = T5Tokenizer


class detoxT5:

    def __init__(self, pretrained ='drive/MyDrive/t5_detox/t5detox.pt', device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
        self.args = Args()
        self.device = device

        model_class, tokenizer_class = self.args.model_class, self.args.tokenizer_class
        self.tokenizer = tokenizer_class.from_pretrained(self.args.model_name_or_path)
        self.model = model_class.from_pretrained(self.args.model_name_or_path)
        if pretrained:
            self.model.load_state_dict(torch.load(pretrained))
            print(f'loading pretrained model from {pretrained}')
        self.model.to(device)


    def train(self,x,y,test_size=0.15, epochs = 5):


        x1, x2, y1, y2 = train_test_split(x, y, test_size=test_size, random_state=42)
        train_dataset = PairsDataset(self.tokenizer(x1), self.tokenizer(y1))
        test_dataset = PairsDataset(self.tokenizer(x2), self.tokenizer(y2))
        data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)


        training_args = Seq2SeqTrainingArguments(
                                  output_dir="bart_detox",
                                  do_train=True,
                                  do_eval=True,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=8,
                                  weight_decay=1e-5,
                                  num_train_epochs=epochs,
                                  learning_rate=3e-5,
                                  evaluation_strategy="steps",
                                  save_strategy="no",
                                  save_total_limit=1,
                                  logging_steps=300,
                                  gradient_accumulation_steps=1,
                              )

        trainer = Seq2SeqTrainer(
                                  model=self.model,
                                  args=training_args,
                                  data_collator = data_collator,
                                  train_dataset=train_dataset,
                                  eval_dataset=test_dataset,
                                  tokenizer=self.tokenizer,
                              )
        trainer.train()

    def paraphrase(self,text, n=None, max_length='auto', temperature=0.0, beams=3):
        texts = [text] if isinstance(text, str) else text
        inputs = self.tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(self.model.device)
        if max_length == 'auto':
            max_length = int(inputs.shape[1] * 1.2) + 10
            result = self.model.generate(
                                            inputs,
                                            num_return_sequences=n or 1,
                                            do_sample=False,
                                            temperature=temperature,
                                            repetition_penalty=3.0,
                                            max_length=max_length,
                                            bad_words_ids=[[2]],
                                            num_beams=beams,
                                        )
        texts = [self.tokenizer.decode(r, skip_special_tokens=True) for r in result]
        if not n and isinstance(text, str):
            return texts[0]
        return texts

